In [6]:
import re
from tqdm import tqdm
import pandas as pd

from utils import random_sample
from utils import add_to_df
import os

In [12]:
ACTION = "CreateLoadsheetAction"

def extract_abcd(df):
    action_df = df[df.action_name == ACTION]
    random_sample(action_df)

    for idx, row in tqdm(action_df.iterrows(), total=action_df.shape[0], desc="Processing rows"):
        current_row = row["entry_details"]

        if "Message type        :   LOADSHEET" or "Message type        :   LOADING_INSTRUCTION" in current_row:
            pattern_edno = r"EDNO (\d+)"
            pattern_takeoff_weight_actual = r'TAKE OFF WEIGHT ACTUAL\s+(\d+)'  # Match one or more digits for takeoff weight
            pattern_all_weights = r'ALL WEIGHTS IN KILOGRAM'  # Exact match for weights unit

            # Extract information using regular expressions
            edno_match = re.search(pattern_edno, current_row)
            takeoff_weight_actual_match = re.search(pattern_takeoff_weight_actual, current_row)
            all_weights_match = re.search(pattern_all_weights, current_row)

            extracted_data = {}
            extracted_data['Loadsheet_Version_Number'] = edno_match.group(1) if edno_match else None
            extracted_data['TAKE_OFF_WEIGHT_ACTUAL'] = takeoff_weight_actual_match.group(1) if takeoff_weight_actual_match else None
            extracted_data['Weights_unit'] = 'KILOGRAM' if all_weights_match else None

            add_to_df(action_df, extracted_data, idx)
        elif "STATUS LOADSHEET" in current_row:
            pass
        elif "STATUS LOADING_INSTRUCTION" in current_row:
            pass
        elif "om.onesystem.lc2.common.dto.SingleAttributeDTO" in current_row:
            pass
        else:
            print(current_row)

    # Ensure the directory exists
    output_dir = "pipeline/actions"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"abcd_{ACTION}.csv")
    return action_df

In [13]:
import pandas as pd
import os
from actions import CalculateWeightAndTrimAction
from actions import StorePaxDataAction
from actions import CreateZFWMessageAction
from actions import CreatLoadSheetAction
from actions import SendLoadsheetAction
from actions import CheckinMsgProcessor
from actions import SendFuelOrderAction
from actions import SetActualBagWeightIndicatorAction
from actions import StoreRegistrationAndConfigurationAc
from actions import UpdateFuelDataAction

# Read the parquet files
# abc_file_path = os.path.join(os.path.dirname(__file__), '../data', 'ABCD_tripfiles.parquet')
current_dir = os.getcwd()
abc_file_path = os.path.join(current_dir, '../data', 'ABCD_tripfiles.parquet')
df_abcd = pd.read_parquet(abc_file_path, engine="pyarrow")

df = pd.DataFrame()
#CalculateWeightAndTrimAction.extract_abcd(df_abcd)
#StorePaxDataAction.extract_abcd(df_abcd)
#CreateZFWMessageAction.extract_abcd(df_abcd)
CreatLoadSheetAction.extract_abcd(df_abcd)
#SendLoadsheetAction.extract_abcd(df_abcd)
#CheckinMsgProcessor.extract_abcd(df_abcd)
#SendFuelOrderAction.extract_abcd(df_abcd)
#SetActualBagWeightIndicatorAction.extract_abcd(df_abcd)
#StoreRegistrationAndConfigurationAc.extract_abcd(df_abcd)
#UpdateFuelDataAction.extract_abcd(df_abcd)

STATUS LOADSHEET 1 LOADING_INSTRUCTION 1 FUEL 1 AIRCRAFT_CONFIG 1 EZFW 2 CARGO_FINAL 1 RAMP_FINAL 1 CARGO_TRANSFER 1 CABIN_CONFIG 1 CALC_HIST_DATA 1 CHECK_IN_OPEN 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 2 REGISTRATION 1 REGISTRATION_CHANGE 4


Processing rows:   0%|          | 0/5055 [00:00<?, ?it/s]c:\Users\Franziska\OneDrive\02-Studium\6. Semester\Projekt\AeroMetrics-1\pipeline\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(key)] = np.nan
c:\Users\Franziska\OneDrive\02-Studium\6. Semester\Projekt\AeroMetrics-1\pipeline\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(key)] = np.nan
c:\Users\Franziska\OneDrive\02-Studium\6. Semester\Projekt\AeroMetrics-1\pipeline\utils.py:12: SettingWithCopyWarning: 
A value i

OSError: Cannot save file into a non-existent directory: 'pipeline\actions\actions_data'